# Team members:

# Muhammed Mamdouh - Ahmed Samir

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge ,Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_log_error as msle
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
%matplotlib inline

# Loading Data

In [ ]:
train=pd.read_csv('../input/bike-sharing-demand/train.csv')
test=pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
train.head()

In [ ]:
test.head()

we have to drop the 2 columns 'casual' and 'registerd' from the training dataset as they are not present in the test dataset

In [ ]:
train.drop(['casual','registered'],1,inplace=True)

# EDA

In [ ]:
train.info()

seems like there is no null values

In [ ]:
train.describe()

seems like there is no abnormal or weird values in the minimum or maximum values

# Data describtion

datetime - hourly date + timestamp

season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 

holiday - whether the day is considered a holiday

workingday - whether the day is neither a weekend nor holiday

weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 

temp - temperature in Celsius

atemp - "feels like" temperature in Celsius

humidity - relative humidity

windspeed - wind speed

casual - number of non-registered user rentals initiated

registered - number of registered user rentals initiated

count - number of total rentals

In [ ]:
#from data describtion we can extract categorcal data and numerical data

In [ ]:
categorical_cols=['season','holiday','workingday','weather']
numerical_cols=['temp','atemp','humidity','windspeed']
label='count'

# Adding timestamp data

adding month,weekday and hour data

In [ ]:
train['datetime'] = pd.to_datetime(train['datetime'])
test['datetime'] = pd.to_datetime(test['datetime'])
train['Month']=pd.DatetimeIndex(train['datetime']).month
test['Month']=pd.DatetimeIndex(test['datetime']).month
train['Year']=pd.DatetimeIndex(train['datetime']).year
test['Year']=pd.DatetimeIndex(test['datetime']).year
train['WeekDay']=pd.DatetimeIndex(train['datetime']).weekday
test['WeekDay']=pd.DatetimeIndex(test['datetime']).weekday
train['Hour']=pd.DatetimeIndex(train['datetime']).hour
test['Hour']=pd.DatetimeIndex(test['datetime']).hour

In [ ]:
categorical_cols.extend(['Month','WeekDay','Hour'])

# Encoding time data ordered by mean of label in each category 

In [ ]:
def encodetime(train,test,col,label):
    d3=train[[col,label]].groupby(col).mean()
    d3.sort_values(by='count',ascending=False)
    plt.scatter(x=d3.index,y=d3['count'])
    d3=d3.sort_values(by='count')
    d3['w']=np.arange(train[col].nunique())
    dic=dict(zip(d3.index,d3['w']))
    train[col]=train[col].map(dic)
    test[col]=test[col].map(dic)

In [ ]:
encodetime(train,test,'Year',label)

In [ ]:
encodetime(train,test,'Month',label)

In [ ]:
encodetime(train,test,'Hour',label)

In [ ]:
encodetime(train,test,'WeekDay',label)

# visualizing Catergorical columns

In [ ]:
def boxplot(x,y,**kwargs):
    sns.boxplot(x=x,y=y)
    x=plt.xticks(rotation=90)
f=pd.melt(train,id_vars=['count'],value_vars=categorical_cols)
g=sns.FacetGrid(f,col='variable',col_wrap=2,sharex=False)
g.map(boxplot,'value','count')

# visualizing numiric coumns

In [ ]:
sns.pairplot(train[[*numerical_cols,'count']])

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
corr = train[[*numerical_cols,'count']].corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

In [ ]:
f, ax = plt.subplots(figsize=(15, 15))
corr = train.corr()
sns.heatmap(corr,cmap=sns.diverging_palette(220, 10, as_cmap=True),square=True, ax=ax, annot = True)

# Transforming Label distribution

In [ ]:
sns.displot(train[label] , kde=True, height=8.27, aspect=11.7/8.27)
sns.displot(np.log(train[label]) , kde=True, height=8.27, aspect=11.7/8.27)

using Log results in left skewed distribution

In [ ]:
def trans(x,l1=0.3,l2=0):
    if l1!=0:
        return ((x+l2)**l1-1)/l1
    else:
        return np.log(x+l2)
def rev_trans(x,l1=0.3,l2=0):
    return (x*l1+1)**(1/l1)-l2

z=train[label].apply(trans)   
sns.displot(z , kde=True, height=8.27, aspect=11.7/8.27)

In [ ]:
#using box cox transform on the label column

In [ ]:
train

# Encoding Weather and Season (One Hot Encoding)

In [ ]:
#train=pd.get_dummies(train,columns=['season','weather'])
#test=pd.get_dummies(test,columns=['season','weather'])

In [ ]:
train

# Spliting Data into Train and Validation

In [ ]:
#removing atemp column as it has high correlation with temp column
x=train.drop(['count','datetime','atemp'],1) 
xtest=test.drop(['datetime','atemp'],1)
y=train['count']
xt,xv,yt,yv=train_test_split(x,y,test_size=0.2,random_state=101)

In [ ]:
#redundunt function .. useful if you decide not to use label transformation function
def redun(x):
    return x

# Model Training and Testing

## Using Random Forest Regressor

In [ ]:
def mk_model_RF(xt1,xv1,yt,yv,md=None,func1=redun,func2=redun,mss=2,n_est=100,al=0):
    
    ytt=yt.apply(func1)
    yvt=yv.apply(func2)
    model=RandomForestRegressor(max_depth=md, random_state=0,min_samples_split=mss,n_estimators=n_est,ccp_alpha=al)
    model.fit(xt1,ytt)
    ypt=np.apply_along_axis(func2,arr=model.predict(xt1),axis=0)
    ypv=np.apply_along_axis(func2,arr=model.predict(xv1),axis=0)
    print('training r2:',r2_score(yt,ypt))
    print('Validation r2:',r2_score(yv,ypv))
    print('training rmsle:',np.sqrt(msle(yt,ypt)))
    print('validation rmsle:',np.sqrt(msle(yv,ypv)))
    return model

In [ ]:
mk_model_RF(xt,xv,yt,yv)#without using label transformation

In [ ]:
mk_model_RF(xt,xv,yt,yv,func1=trans,func2=rev_trans,n_est=400,md=20)#using label transformation

## Using XGradient Boosting Regressor

In [ ]:
def mk_model_xgb(xt,xv,yt,yv,func1=redun,func2=redun,lr=1,min_child_weight =25,colsample_bytree = 0.8,md=None):
    model =XGBRegressor( colsample_bytree = colsample_bytree, learning_rate = lr,min_child_weight =min_child_weight, max_depth=md )
    ytt=yt.apply(func1)
    model.fit(xt,ytt)
    ypt=np.apply_along_axis(func2,arr=model.predict(xt),axis=0)
    ypv=np.apply_along_axis(func2,arr=model.predict(xv),axis=0)
    print('training r2:',r2_score(yt,ypt))
    print('Validation r2:',r2_score(yv,ypv))
    print('training rmsle:',np.sqrt(msle(yt,ypt)))
    print('validation rmsle:',np.sqrt(msle(yv,ypv)))
    return model

In [ ]:
_=mk_model_xgb(xt,xv,yt,yv,func1=trans,func2=rev_trans,lr=0.2,min_child_weight =20,colsample_bytree = 0.8,md=20)

XGradient Boosting Regressor yielded better results so we trained the model with whole data using these parameters

In [ ]:
model=XGBRegressor(colsample_bytree = 0.8, learning_rate = 0.2,min_child_weight =20, max_depth=20).fit(x,y.apply(trans))

## using label transformation gives better rmsle

In [ ]:
xtest

In [ ]:
model.predict(xtest)

# Applying inverse transformation

In [ ]:
yp=np.apply_along_axis(rev_trans,arr=model.predict(xtest),axis=0)

# prediction Distripution

In [ ]:
plt.hist(yp)

In [ ]:
test['count']=yp

In [ ]:
test

# Produce Submission File

In [ ]:
test[['datetime', 'count']].to_csv('/kaggle/working/submission.csv', index=False)